- Author: Benjamin Du
- Date: 2023-03-25 14:48:30
- Modified: 2023-03-25 14:48:30
- Title: Manipulate Pull Requests Using GitHub Rest APIs
- Slug: manipulate-pull-requests-using-github-REST-APIs
- Category: Computer Science
- Tags: Computer Science, programming, GitHub, REST, RESTful, API, Python, pull request

**Things on this page are fragmentary and immature notes/thoughts of the author. Please read with your own judgement!**

In [59]:
import json
import requests


def list_pull_requests(token: str, owner: str, repo: str):
    return requests.get(
        url=f"https://api.github.com/repos/{owner}/{repo}/pulls",
        headers={
            "Accept": "application/vnd.github.v3+json",
            "Authorization": f"token {token}",
        },
    )


def create_pull_request(token: str, owner: str, repo: str, data: dict[str, str]):
    """Create a pull request.
    Notice that the error code 422 (Unprocessable Entity)
    means there's no changes to merge from the head branch to the base branch,
    so it is not really an error.

    :param token: The personal access token for authentication.
    :param owner: Owner of the repository.
    :param repo: Name of the repository.
    :param data: A dict containing information (e.g., base, head, title, body, etc.)
    about the pull request to be created.
    """
    headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {token}",
    }
    return requests.post(
        url=f"https://api.github.com/repos/{owner}/{repo}/pulls",
        headers=headers,
        data=json.dumps(data)
    )


def merge_pull_request(token: str, owner: str, repo: str, pr_number: int):
    headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {token}",
    }
    return requests.put(
        url=f"https://api.github.com/repos/{owner}/{repo}/pulls/{pr_number}/merge",
        headers=headers,
    )